# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [103]:
# import libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import sqlite3
import re
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import PorterStemmer

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import Normalizer
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import GridSearchCV

nltk.download(['punkt', 'wordnet','stopwords','averaged_perceptron_tagger'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [19]:
# load data from database
engine = create_engine('sqlite:///DisasterTable.db')
df = df = pd.read_sql("SELECT * FROM DisasterTable", engine)
df.nunique()

id                        26180
message                   26177
original                   9630
genre                         3
related                       3
request                       2
offer                         2
aid_related                   2
medical_help                  2
medical_products              2
search_and_rescue             2
security                      2
military                      2
child_alone                   1
water                         2
food                          2
shelter                       2
clothing                      2
money                         2
missing_people                2
refugees                      2
death                         2
other_aid                     2
infrastructure_related        2
transport                     2
buildings                     2
electricity                   2
tools                         2
hospitals                     2
shops                         2
aid_centers                   2
other_in

In [20]:
df.isnull().sum()

id                            0
message                       0
original                  16046
genre                         0
related                       0
request                       0
offer                         0
aid_related                   0
medical_help                  0
medical_products              0
search_and_rescue             0
security                      0
military                      0
child_alone                   0
water                         0
food                          0
shelter                       0
clothing                      0
money                         0
missing_people                0
refugees                      0
death                         0
other_aid                     0
infrastructure_related        0
transport                     0
buildings                     0
electricity                   0
tools                         0
hospitals                     0
shops                         0
aid_centers                   0
other_in

In [22]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
X = df['message']
Y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)

In [25]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [26]:
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
dic={}
for col in Y.columns:
    dic[col] = Y[col].sum()
dic

{'related': 20282,
 'request': 4474,
 'offer': 118,
 'aid_related': 10860,
 'medical_help': 2084,
 'medical_products': 1313,
 'search_and_rescue': 724,
 'security': 471,
 'military': 860,
 'child_alone': 0,
 'water': 1672,
 'food': 2923,
 'shelter': 2314,
 'clothing': 405,
 'money': 604,
 'missing_people': 298,
 'refugees': 875,
 'death': 1194,
 'other_aid': 3446,
 'infrastructure_related': 1705,
 'transport': 1201,
 'buildings': 1333,
 'electricity': 532,
 'tools': 159,
 'hospitals': 283,
 'shops': 120,
 'aid_centers': 309,
 'other_infrastructure': 1151,
 'weather_related': 7297,
 'floods': 2155,
 'storm': 2443,
 'fire': 282,
 'earthquake': 2455,
 'cold': 530,
 'other_weather': 1376,
 'direct_report': 5075}

### 2. Write a tokenization function to process your text data

In [29]:
def tokenize(text):
    text = text.lower()
    text = re.sub(r"[^a-zA-z0-9]"," ",text)
    words = word_tokenize(text)
    words = [w for w in words if w not in stopwords.words("english")]
    
    clean_tokens = []
    lemmatizer = WordNetLemmatizer()
    
    for w in words:
        clean_tok = lemmatizer.lemmatize(w , pos='v').strip()
        clean_tokens.append(clean_tok)
    return clean_tokens


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [30]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer(use_idf=True)),
    ('clf', MultiOutputClassifier(AdaBoostClassifier())),
]) 

In [34]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f4fcd029510>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
             learning_rate=1.0, n_estimators=50, random_state=None),
              n_jobs=1))],
 'vect': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_features=None, min_df=1,
         ngram_rang

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [40]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y,test_size = 0.2, random_state = 45)
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [41]:
Y_pred = pipeline.predict(X_test)

In [81]:
def display_results(y_test, y_pred, y_col):
    """
        Display f1 score, precision, recall, accuracy and confusion_matrix
        for each category of the test dataset
    """

    clf_report = classification_report(y_test, y_pred)
    accuracy = (y_pred == y_test).mean()
    print(y_col, ":")
    print('\n')
    print(clf_report)
    print('Accuracy =', accuracy)
    print('-'*60)
    print('\n')

In [82]:
col=0
cols = Y_test.columns
for categorie in Y_test.columns:
    display_results(Y_test[categorie],
                    Y_pred[:,col],
                    categorie)
    col+=1

related :


             precision    recall  f1-score   support

          0       0.60      0.16      0.25      1198
          1       0.79      0.97      0.87      4002
          2       0.45      0.11      0.18        44

avg / total       0.74      0.78      0.72      5244

Accuracy = 0.775553012967
------------------------------------------------------------


request :


             precision    recall  f1-score   support

          0       0.91      0.97      0.94      4335
          1       0.77      0.52      0.62       909

avg / total       0.88      0.89      0.88      5244

Accuracy = 0.890350877193
------------------------------------------------------------


offer :


             precision    recall  f1-score   support

          0       0.99      1.00      1.00      5214
          1       0.00      0.00      0.00        30

avg / total       0.99      0.99      0.99      5244

Accuracy = 0.993325705568
------------------------------------------------------------


a

### 6. Improve your model
Use grid search to find better parameters. 

In [85]:
parameters = {

    'tfidf__use_idf': [True, False],
    'clf__estimator__learning_rate': [0.5, 0.9],
    'clf__estimator__n_estimators': [50, 100]

}

cv = GridSearchCV(pipeline, param_grid=parameters,
                  cv=2, n_jobs=-1, verbose=2)

cv.fit(X_train, Y_train) 

cv.best_params_

Fitting 2 folds for each of 8 candidates, totalling 16 fits
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=50, tfidf__use_idf=True 
[CV]  clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=50, tfidf__use_idf=True, total= 2.3min
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=50, tfidf__use_idf=True 


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  3.1min remaining:    0.0s


[CV]  clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=50, tfidf__use_idf=True, total= 2.3min
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=50, tfidf__use_idf=False 
[CV]  clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=50, tfidf__use_idf=False, total= 2.2min
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=50, tfidf__use_idf=False 
[CV]  clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=50, tfidf__use_idf=False, total= 2.2min
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=100, tfidf__use_idf=True 
[CV]  clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=100, tfidf__use_idf=True, total= 3.1min
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=100, tfidf__use_idf=True 
[CV]  clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=100, tfidf__use_idf=True, total= 3.1min
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=10

[Parallel(n_jobs=-1)]: Done  16 out of  16 | elapsed: 55.4min finished


{'clf__estimator__learning_rate': 0.9,
 'clf__estimator__n_estimators': 100,
 'tfidf__use_idf': False}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [89]:
cv.best_estimator_

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ator=None,
          learning_rate=0.9, n_estimators=100, random_state=None),
           n_jobs=1))])

In [91]:
Y_pred = cv.predict(X_test)

In [94]:
(Y_test == Y_pred).mean().mean()

0.94743198576150511

In [ ]:
col=0
cols = Y_test.columns
for categorie in Y_test.columns:
    display_results(Y_test[categorie],
                    Y_pred[:,col],
                    categorie)
    col+=1

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [118]:
def tokenize_2(text):
    """
        Tokenize the input text. This function is called in StartingVerbExtractor.
    """

    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    clean_tokens = [lemmatizer.lemmatize(
        tok).lower().strip() for tok in tokens]

    return clean_tokens


class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        """ return true if the first word is an appropriate verb or RT for retweet """
        # tokenize by sentences
        sentence_list = nltk.sent_tokenize(text)

        for sentence in sentence_list:
            # tokenize each sentence into words and tag part of speech
            pos_tags = nltk.pos_tag(tokenize_2(sentence))
            # index pos_tags to get the first word and part of speech tag
            first_word, first_tag = pos_tags[0]

            # return true if the first word is an appropriate verb or RT for retweet
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, x, y=None):
        """ Fit """
        return self

    def transform(self, X):
        """ Transform """
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [119]:
pipeline_random = Pipeline([
    ('features', FeatureUnion([

        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer(use_idf=True))
        ])),

        ('start_verb', StartingVerbExtractor())
    ])),

    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

In [120]:
pipeline_random.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [121]:
Y_pred = pipeline_random.predict(X_test)

In [122]:
(Y_pred == Y_test).mean().mean()

0.94375052970590723

### Try AdaBoostClassifier

In [124]:
pipeline_ada = Pipeline([
    ('features', FeatureUnion([

        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer(use_idf=True))
        ])),

        ('start_verb', StartingVerbExtractor())
    ])),

    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])

pipeline_ada.fit(X_train, Y_train)
Y_pred = pipeline_ada.predict(X_test)
(Y_pred == Y_test).mean().mean()

0.94639905924230883

In [126]:
col=0
cols = Y_test.columns
for categorie in Y_test.columns:
    display_results(Y_test[categorie],
                    Y_pred[:,col],
                    categorie)
    col+=1

related :


             precision    recall  f1-score   support

          0       0.60      0.16      0.25      1198
          1       0.79      0.97      0.87      4002
          2       0.45      0.11      0.18        44

avg / total       0.74      0.78      0.72      5244

Accuracy = 0.775553012967
------------------------------------------------------------


request :


             precision    recall  f1-score   support

          0       0.91      0.97      0.94      4335
          1       0.77      0.52      0.62       909

avg / total       0.88      0.89      0.88      5244

Accuracy = 0.890350877193
------------------------------------------------------------


offer :


             precision    recall  f1-score   support

          0       0.99      1.00      1.00      5214
          1       0.00      0.00      0.00        30

avg / total       0.99      0.99      0.99      5244

Accuracy = 0.992753623188
------------------------------------------------------------


a

### 9. Export your model as a pickle file

In [129]:
import pickle
pickle.dump(pipeline_ada,open('./models/model_adaboost','wb'))

In [130]:
import pickle
pickle.dump(pipeline_random,open('./models/model_random','wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.